In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(hise)
quiet_library(dplyr)
quiet_library(purrr)

## Read GSEA results stored in HISE

In [2]:
dap_uuid <- list("a8973524-cc71-40d5-8edf-3d6a9e0a77d4")

In [3]:
dap_fres <- cacheFiles(dap_uuid)
dap_file <- list.files(
    paste0("cache/",dap_uuid),
    recursive = TRUE, full.names = TRUE
)

submitting request as query ID first...

retrieving files using fileIDS...



In [4]:
all_dap <- read.csv(dap_file)

## Format for output

In [5]:
names(all_dap)

[1] "aifi_cell_type" "fg"             "bg"             "seqnames"      
 [5] "start"          "end"            "logFC"          "adjP"          
 [9] "MeanDiff"       "idx"

In [6]:
all_dap <- all_dap %>%
  mutate(timepoint = sub(".+_","",fg),
         fg = sub("_.+","",fg),
         bg = sub("_.+","",bg)) %>%
  rename(chr = seqnames) %>%
  select(-idx)

In [7]:
nrow(all_dap)

[1] 3281208

Filter peaks with high FDR to reduce file sizes

In [8]:
fdr_cutoff <- 0.2

out_dap <- all_dap %>%
  filter(adjP < fdr_cutoff)

nrow(out_dap)

[1] 117459

In [9]:
out_dir <- "output"
if(!dir.exists("output")) { dir.create("output") }

In [10]:
out_file <- paste0("Supp-Table-05_ArchR_DAPs_", Sys.Date(), ".csv")
write.csv(
    out_dap,
    file.path(out_dir, out_file),
    row.names = FALSE,
    quote = FALSE
)

In [11]:
system(paste('gzip', file.path(out_dir, out_file)))

In [12]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] purrr_1.0.2 dplyr_1.1.3 hise_2.16.0

loaded via a namespace (and not attached):
 [1] crayon_1.5.2     vctrs_0.6.3      httr_1.4.7       cli_3.6.1       
 [5] rlang_1.1.1      generics_0.1.3   assertthat_0.2.1 jsonlite_1.8.7  
 [9] glue_1.6.2       RCurl_1.98-1.12  htmltools_0.5.6  IRdisplay_1.1   
[13] IRker